2.4 Extension

Further, notice how the bright regions in the Fourier-domain image are concentrated in a small set of frequency components. 

In [ ]:
fourier_img = fourier_transform(img.flatten(), Q)
fourier_img = M_fourier @ S_fourier @ fourier_img  # Feel free to experiment here!
pixel_img = inv_fourier_transform(fourier_img, Q)

plt.subplot(121)
plt.imshow(fourier_img.reshape(img.shape), cmap="gray")
plt.title("Fourier domain")
plt.axis("off")

plt.subplot(122)
plt.imshow(pixel_img.reshape(img.shape), cmap="gray")
plt.title("Pixel domain")
plt.axis("off")
plt.show()

3.2 Extension

Implementing a numerically stable second-order Gaussian filter can be complicated. Compare your implementation, which should return a constant zero-valued second-derivative in the y-direction, with a package that implements Gaussian filters incorrectly... `scipy`!

`scipy`'s implementation has errors of magnitude $10^3$, while yours should be < $10^{-13}$.

Why does this occur? This discrepancy occurs because computing higher-order derivatives of Gaussian filters is numerically sensitive, especially when the filter is truncated to a finite length. In theory, the second derivative of a linear function is exactly zero. Since the test image is constructed so that its second derivative in the $y$-direction is zero, an ideal Gaussian second-derivative filter should produce values very close to zero everywhere. However, in practice, Gaussian filters must be sampled and truncated. If derivatives are computed by repeatedly applying finite-difference or discrete differentiation rules, small approximation errors are introduced. These errors accumulate and are amplified when taking higher-order derivatives, leading to large numerical inaccuracies.

In [ ]:
# Construct an image with a zero-valued second-derivative in the column-direction
img = torch.zeros(1, 1, 31, 31, dtype=torch.float64)
img[..., 7:24] += torch.arange(1, 18)
img[..., 24:] += 17

for sigma in [0.5]: # Modify as desired
    # Apply your second-derivative filter
    our_second_derivative = second_order_yy(img, sigma=sigma)

    # Apply the second-derivative filter using scipy
    scipy_second_derivative = gaussian_filter_scipy(
        img.numpy().squeeze(), sigma=sigma, order=[2, 0]
    )

    # Plot the original image and the filtered images
    plt.figure(figsize=(15, 3.75))
    plt.subplot(131)
    plt.imshow(img.squeeze(), cmap="gray")
    plt.title("Original Image")
    plt.colorbar()

    plt.subplot(132)
    plt.imshow(our_second_derivative.squeeze(), cmap="turbo")
    plt.title("Your Second Derivative")
    plt.colorbar()

    plt.subplot(133)
    plt.imshow(scipy_second_derivative, cmap="turbo")
    plt.title("Scipy Second Derivative")
    plt.colorbar()

    plt.suptitle(f"Sigma = {sigma}")
    plt.tight_layout()
    plt.show()

In [ ]:
sigma = 1
fig, axs = imshow(
    img := astronaut(),
    zeroth_order(img, sigma),
    first_order_x(img, sigma),
    first_order_y(img, sigma),
    second_order_xx(img, sigma),
    second_order_yy(img, sigma),
)
axs[0].set_title("Original image")
axs[1].set_title("Smoothed")
axs[2].set_title("First x-derivative")
axs[3].set_title("First y-derivative")
axs[4].set_title("Second x-derivative")
axs[5].set_title("Second y-derivative")
plt.show()

3.3 Extension

In [ ]:
sigma = 1
fig, axs = imshow(
    img := astronaut(),
    log(img, sigma),
)
axs[0].set_title("Original image")
axs[1].set_title("LoG filtered image")
plt.show()

3.4 Extension

In [ ]:
# Verify your 2D filters are properly oriented
fig, axs = plt.subplots(2, 4, figsize=(8, 4))
for idx, ax in enumerate(axs.flatten()):
    ax.imshow(oriented_filter(idx * torch.pi / 4, 2.5), cmap="bwr")
    ax.axis("off")
    ax.set_title(f"$\\theta = {idx} \\pi / 4$")
plt.tight_layout()
plt.show()